# 1. Objective

To fit models for repeated measures data

Repeated measures data can thought of to exist in two levels. The subject level (level 2) and the unit of analysis being the repeated measurements on each subject (level 1). Covariates at level 2 describe the between-subjects variation and covariates at level 1 describe the within-subjects variation.

In [4]:
data_df = read.delim("data/rat_brain.dat.txt")

In [5]:
glimpse(data_df)

Observations: 30
Variables: 4
$ animal    <fct> R111097, R111097, R111097, R111097, R111097, R111097, R1113…
$ treatment <int> 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1,…
$ region    <int> 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2,…
$ activate  <dbl> 366.19, 199.31, 187.11, 371.71, 302.02, 449.70, 375.58, 204…


Formally, we allow the intercept and slope of treatment to vary by observations on animals:

For observation $t$ on animal $i$:

$$ activate_{ti} =  \beta_0 + \beta_1 region1_{ti} + \beta_2 region2_{ti} + \beta_3 treatment_{ti} + \\
                     \beta_4 region1_{ti} \times treatment_{ti} + \beta_5 region2_{ti} \times treatment_{ti} + \\
                      u_{0i} + u_{3i} treatment_{ti} + \epsilon_{ti}$$ 
                      
$$ u_i = \begin{bmatrix} u_{0i} \\ u_{3i} \end{bmatrix} \sim N(0, \mathbf{D})$$

$$ \epsilon_i = \begin{bmatrix} \epsilon_{1i} \\ \vdots \\ \epsilon_{6i} \end{bmatrix} \sim N(0, \sigma^2I_6)$$

In [6]:
data_df %>%
    mutate(treatment = ifelse(treatment == 1, 0, 1),
           region = case_when(region == 1 ~ 1,
                              region == 2 ~ 2,
                              region == 3 ~ 0)) %>%
    mutate(region = factor(region)) ->
    data_df

In [7]:
glimpse(data_df)

Observations: 30
Variables: 4
$ animal    <fct> R111097, R111097, R111097, R111097, R111097, R111097, R1113…
$ treatment <dbl> 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,…
$ region    <fct> 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2,…
$ activate  <dbl> 366.19, 199.31, 187.11, 371.71, 302.02, 449.70, 375.58, 204…


In [9]:
base_model = lme(activate ~ region * treatment,
                 random = ~ 1 | animal,
                 data = data_df,
                 method = "REML")

In [13]:
proposed_model1 = lme(activate ~ region * treatment,
                     random = ~ 1 + treatment | animal,
                     data = data_df,
                     method = "REML")

In [14]:
anova(base_model, proposed_model1)

,call,Model,df,AIC,BIC,logLik,Test,L.Ratio,p-value
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>
base_model,"lme.formula(fixed = activate ~ region * treatment, data = data_df, random = ~1 | animal, method = ""REML"")",1,8,291.2822,300.7066,-137.6411,,NA,NA
proposed_model1,"lme.formula(fixed = activate ~ region * treatment, data = data_df, random = ~1 + treatment | animal, method = ""REML"")",2,10,269.1904,280.9710,-124.5952,1 vs 2,26.09171,2.159021e-06


In [17]:
proposed_model2 = lme(activate ~ region * treatment,
                      random = ~ 1 + treatment | animal,
                      weights = varIdent(form = ~ 1 | treatment),
                      data = data_df,
                      method = "REML")

In [18]:
anova(proposed_model1, proposed_model2)

,call,Model,df,AIC,BIC,logLik,Test,L.Ratio,p-value
,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>
proposed_model1,"lme.formula(fixed = activate ~ region * treatment, data = data_df, random = ~1 + treatment | animal, method = ""REML"")",1,10,269.1904,280.9710,-124.5952,,NA,NA
proposed_model2,"lme.formula(fixed = activate ~ region * treatment, data = data_df, random = ~1 + treatment | animal, weights = varIdent(form = ~1 | treatment), method = ""REML"")",2,11,271.0383,283.9969,-124.5192,1 vs 2,0.1521269,0.696511


In [19]:
summary(proposed_model1)

Linear mixed-effects model fit by REML
 Data: data_df 
       AIC     BIC    logLik
  269.1904 280.971 -124.5952

Random effects:
 Formula: ~1 + treatment | animal
 Structure: General positive-definite, Log-Cholesky parametrization
            StdDev   Corr  
(Intercept) 35.83696 (Intr)
treatment   79.82012 0.801 
Residual    23.21432       

Fixed effects: activate ~ region * treatment 
                     Value Std.Error DF    t-value p-value
(Intercept)        212.294  19.09551 20  11.117483  0.0000
region1            216.212  14.68203 20  14.726304  0.0000
region2             25.450  14.68203 20   1.733412  0.0984
treatment          360.026  38.59808 20   9.327561  0.0000
region1:treatment -261.822  20.76352 20 -12.609710  0.0000
region2:treatment -162.500  20.76352 20  -7.826225  0.0000
 Correlation: 
                  (Intr) regin1 regin2 trtmnt rgn1:t
region1           -0.384                            
region2           -0.384  0.500                     
treatment          0.4

In [ ]:
intervals(proposed_model1)

Approximate 95% confidence intervals

 Fixed effects:
                        lower     est.      upper
(Intercept)        172.461465  212.294  252.12653
region1            185.585828  216.212  246.83817
region2             -5.176172   25.450   56.07617
treatment          279.511807  360.026  440.54019
region1:treatment -305.133948 -261.822 -218.51005
region2:treatment -205.811948 -162.500 -119.18805
attr(,"label")
[1] "Fixed effects:"

 Random Effects:
  Level: animal 
                                lower       est.       upper
sd((Intercept))            16.2424580 35.8369586  79.0697815
sd(treatment)              38.3782028 79.8201157 166.0122261
cor((Intercept),treatment) -0.3464772  0.8009907   0.9882172

 Within-group standard error:
   lower     est.    upper 
16.41655 23.21432 32.82691 

# R Environment

In [8]:
library(tidyverse)
library(nlme)

In [2]:
setwd("~/mixed-effects-models")

In [3]:
print(sessionInfo())

R version 3.6.1 (2019-07-05)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/libopenblasp-r0.2.20.so

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] nlme_3.1-141    forcats_0.4.0   stringr_1.4.0   dplyr_0.8.0.1  
 [5] purrr_0.3.2     readr_1.3.1     tidyr_0.8.3     tibble_2.1.1   
 [9] ggplot2_3.1.1   tidyverse_1.2.1

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.1          cellranger_1.1.0    plyr_1.8.4         
 [4] pillar_1.3.1        compiler_3.6.1     